# Inversion example of diffusion + tracer mineral (uranium salt) dissolution inversion in 2d

- Importation of the required modules

In [ ]:
import copy
import os
from pathlib import Path
from typing import Sequence, Tuple

import gstools as gs
import matplotlib as mpl
import matplotlib.pyplot as plt
import nested_grid_plotter as ngp
import numpy as np
import pandas as pd
import pyrtid
from IPython.display import HTML
from matplotlib.animation import HTMLWriter
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon

- Check package/software versions

In [ ]:
pyrtid.utils.show_versions()

- Create a directory to store the exported figures

In [ ]:
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
fig_save_path = Path(ipynb_path, "exported_figures")
fig_save_path.mkdir(parents=True, exist_ok=True)  # make sure that the directory exists

- Define some configurations for the plots

In [ ]:
# Some configs for the plots
new_rc_params = {
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica", "DejaVu Sans"],
    "font.size": 16,
    "text.usetex": False,
    "savefig.format": "svg",
    "svg.fonttype": "none",  # to store text as text, not as path
    "savefig.facecolor": "w",
    "savefig.edgecolor": "k",
    "savefig.dpi": 300,
    "figure.constrained_layout.use": True,
    "figure.facecolor": "w",
    "axes.facecolor": "w",
}
csfont = {"fontname": "Comic Sans MS"}
hfont = {"fontname": "Helvetica"}
plt.rcParams.update(new_rc_params)

- Define a runner

In [ ]:
if runner_type == RunnerType.FRONTAL:
    runner = FrontalHytecRunner(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        nb_cpu=4,
        freq_checks_is_simu_over_sec=5,
    )
elif runner_type == RunnerType.SLURM:
    job_config = JobConfig(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        nb_nodes=1,
        ncpus=12,
        queue="geo-cpu",
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        dos2unix_binary_path_or_alias=dos2unix_binary_path_or_alias,
    )
    runner = SlurmHytecRunner(
        job_config=job_config,
        freq_checks_is_simu_over_sec=5,
    )
elif runner_type == RunnerType.QSUB:
    job_config = JobConfig(
        hytec_binary_path_or_alias=hytec_binary_path_or_alias,
        nb_nodes=1,
        ncpus=12,
        queue="Omines_cpu",
        mpi_binary_path_or_alias=mpi_binary_path_or_alias,
        dos2unix_binary_path_or_alias=dos2unix_binary_path_or_alias,
    )
    runner = QsubHytecRunner(
        job_config=job_config,
        freq_checks_is_simu_over_sec=5,
    )
else:
    raise Exception("Could not created runner")

## Forward problem

- Creation of a base simulation

In [ ]:
# Create an empty simulation in a non existing folder
simu_base = HytecSimulation("simu_base", Path.cwd().joinpath("simu_base"))

htc = simu_base.handlers.htc
htc_path = simu_base.htc_file_path

# Add a TDB file
simu_base.link_tdb("./../../../TDB/chess.tdb")  # This is relative to the htc file

htc.output_format = "vtk"

# Hydrodynamic parameters definition
htc.flow_regime = "stationary"
htc.porosity = 0.23
htc.permeability = 8e-5
htc.permeability_units = "m/s"
htc.diffusion_coefficient = 1e-6
htc.diffusion_coefficient_units = "m2/s"
htc.head = 0

# Boundary definition
nx = 57
ny = 57
nz = 1
dx = 5
dy = 5
dz = 1
htc.grid_regime = "rectangle"
htc.domain = f"{nx*dx},{nx} {ny*dy},{ny}"

# Geometry
dzone = HytecZone("domain", htc_path)
dzone.geometries = ["domain"]
dzone.geochem = "chem_base"
htc.zones_dict["domain"] = dzone

# Boundary conditions
left_border = HytecBoundary("border_left", htc_path)
left_border.coordinates = f"0,0, 0,{ny*dy} m"
htc.boundaries_dict["left"] = left_border

right_border = HytecBoundary("border_right", htc_path)
right_border.coordinates = f"{nx*dx},{ny*dy}, {nx*dx},0 m"
left_border.flow_condition = "constant-head at 0 m"
htc.boundaries_dict["right"] = right_border

top_border = HytecBoundary("border_top", htc_path)
top_border.coordinates = f"0,{ny*dy}, {nx*dx},{ny*dy} m"
htc.boundaries_dict["top"] = top_border

bottom_border = HytecBoundary("border_bottom", htc_path)
bottom_border.coordinates = f"0,0, {nx*dx},0 m"
htc.boundaries_dict["bottom"] = bottom_border

for border in htc.boundaries_dict.values():
    border.flow_condition = "constant-head at 0 m"

# Geochemical unit definition
htc.report = "full"
htc.redox = "enabled"
new_unit = HytecGeochemUnit("chem_base")
new_unit.concentrations["T_Cinet"] = Concentration("T_Cinet", 0.0, "molal")
new_unit.minerals["Min_T_Cinet"] = Mineral(
    "Min_T_Cinet", 0.0, grade_units="mol/kg", surface=13.5, surface_units="m2/mol"
)
htc.geochem_units_dict[new_unit.name] = new_unit

# Time specification
dt = 800
duration_in_days = 5
nsamples = 30
htc.duration = duration_in_days * 3600 * 24
htc.duration_units = "s"
htc.timestep = (
    f"variable {{\n\tstart = {dt} s\n\tmaximum = {dt} s\n\tcourant-factor = 20\n}}"
)
htc.samples = nsamples

# define, extend
new_tracer = HytecDefinition(
    phase="basis", species="T_Cinet", content="""\n\tmoleweight = 270 g/mol\n"""
)
htc.define_list.append(new_tracer)

new_mineral = HytecDefinition(
    species="Min_T_Cinet",
    phase="mineral",
    content="""
	composition = 1 T_Cinet
	logK = 3.12
	surface = 500 cm2/g
	kinetics {
		rate = -6.9e-9 mol/m2/s
		area = Min_T_Cinet
		y-term, species = Min_T_Cinet
	}\n""",
)
htc.define_list.append(new_mineral)

# select
htc.verbose = "enabled"
htc.grid_select_list = [
    "time in s",
    "node-number",
    # "diffusion",
    # "permeability",
    "head",
    "x-flowrate",
    "y-flowrate",
    "Min_T_Cinet in mol/kg",
    "T_Cinet in mol/l",
]

# Careful -> do not put units otherwise a factor 1000 is introduced (30/06/2022)
htc.flux_select_list = ["T_Cinet"]

# exclude
htc.exclude_list = ["minerals", "colloids", "gases"]

- Creation of a network of injectors and producers in shape of hexagons.

In [ ]:
# locations in the grid
# selection = index of cells to select. Cells are sorted by y, and if that's equal by x (increasing order).
cell_radius = 40.0
inj_grid_locations, prod_grid_locations, polygons = gen_wells_coordinates(
    -25.0,
    65.0,
    175.0,
    235.0,
    radius=cell_radius,
    rotation=-30,
    selection=[0, 1, 2, 4, 5, 6, 7],
)

In [ ]:
plotter = FieldPlotter(fig_params={"figsize": (8, 7)})

for i, j in inj_grid_locations:
    plotter.ax_dict["ax1-1"].plot(i, j, "ko")

for i, j in prod_grid_locations:
    plotter.ax_dict["ax1-1"].plot(i, j, "ro")

patches = []
for polygon in polygons:
    # Add the polygon to the collection of patches
    xy = np.array(polygon)
    patches.append(Polygon(xy, closed=True, facecolor=None))
    # Plot the number of the polygon
p = PatchCollection(patches, alpha=0.75)
p.set_facecolors("white")
p.set_edgecolors("darkgrey")
p.set_linewidth(0.6)
p.set_linestyle("-")
plotter.ax_dict["ax1-1"].add_collection(p)

for i, prod_coords in enumerate(prod_grid_locations):
    plotter.ax_dict["ax1-1"].text(*prod_coords, i)

plotter.ax_dict["ax1-1"].set_aspect("equal", adjustable="box")
plotter.ax_dict["ax1-1"].set_xlim(0, nx * dx)
plotter.ax_dict["ax1-1"].set_ylim(0, ny * dy)
plotter.ax_dict["ax1-1"].set_xlabel("X (m)", fontweight="bold")
plotter.ax_dict["ax1-1"].set_ylabel("Y (m)", fontweight="bold")

fname = "7_cells_wellfield"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Create two models from this base simulation

In [ ]:
simu_true = copy.deepcopy(simu_base)
simu_true.update_root_and_name(new_root="simu_true", new_name="simu_true")
simu_estimate = copy.deepcopy(simu_base)
simu_estimate.update_root_and_name(new_root="simu_estimate", new_name="simu_estimate")

The mineral grades are defined in [mol/kg]. We calculate the conversion factor to obtain ppm and perform easier mass balances.

ConvU: parameter for converting the mineral content in [mol/kg] to metal grade in [ppm]. Note that the ConvU parameter is specific to the Uranium carrier phase: here Uraninite.

$C_{Uraninite}[\frac{mol}{kg}] = convU \times T_{Uranium}[ppm]$


and


$conv_u = \frac{1.023 \times density_{rock}}{238*porosity_{rock} \times 1000}$

Note the 1.023 is the conversion factor from molar mass to molal mass in CHESS.

In [ ]:
rock_density = 1.63
conv_u: np.ndarray = (
    1.023 * rock_density / (238.0 * simu_base.handlers.htc.porosity * 1000)
)

- Two different Min_T_Cinet spatial distributions

In [ ]:
# Create a Gaussian Covariance Model just for the example
# To vary the results, change the seed :)
seed = 194

min_val = 32 * conv_u  # 32 ppm
max_val = 700 * conv_u  # 700 ppm
# Compute the mean and the standard deviation that the distribution should have so that
# <99% of the values are between min and max ~ 6 sigmas
mean = (max_val + min_val) / 2
stdev = (max_val - min_val) / 2 / 3  # std ~ 1/6 of the distribution interval
len_scale = 10

true_min_t_cinet = gen_random_ensemble(
    model=gs.covmodel.Gaussian,
    n_ensemble=1,
    var=stdev**2,
    len_scale=len_scale,
    mean=mean,
    nx=nx,
    ny=ny,
    seed=seed,
)[0, :, :, 0]

# Initial estimate = an homogeneous value
estimate_min_t_cinet = np.ones((nx, ny)) * 200 * conv_u  # 200 ppm

plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 6)},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True)
    },
)
plotter.plot_2d_field(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "True": true_min_t_cinet / conv_u,
        "Estimated": estimate_min_t_cinet / conv_u,
    },
    cbar_title="Metal grade $[ppm]$",
    imshow_kwargs={"cmap": plt.get_cmap("jet"), "extent": [0.0, nx * dx, 0.0, ny * dy]},
    xlabel="X (m)",
    ylabel="Y (m)",
)
plotter.subfigs["fig0"].suptitle("Metal grade [ppm]", fontweight="bold")

fname = "Min_T_Cinet_field_true_vs_initial_estimation"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Plot the reserves per mesh

In [ ]:
def get_cell_reserves(
    u_field_grades: NDArrayFloat,
    cell_polygons: Sequence[Sequence[Tuple[float, float]]],
    dx: float,
    dy: float,
    dz: float,
) -> Sequence[float]:
    """Return the reserves in t under the given technological cell.

    Parameters
    ----------
    u_field_grades : NDArrayFloat
        Uraninite grades field in ppm. 2D Array.
    dx : float
        X dimension of one mesh (m).
    dy : float
        Y dimension of one mesh (m).
    dz : float
        Z dimension of one mesh (m).
    mask: NDArrayFloat
        Mask to apply before doing the selection.
    Returns
    -------
    float
        The associated uranium reserves in t.
    """
    # flatten points coordinates
    _x, _y, _z = hytec_node_number_to_indices(np.arange(nx * ny), nx, ny)
    _x = _x * dx + dx / 2
    _y = _y * dy + dy / 2
    grid_coords = np.array((_x, _y)).T

    cell_volume = dx * dy * dz  # m3
    # Conversion factor to go from a number of ppm per unit of volume to a mass in t (all cells are the same)
    conv_factor = cell_volume * rock_density * 1e-6 / conv_u

    reserves_list = []

    mask_sum = None
    for cell_polygon in cell_polygons:
        # Select the mesh that belongs to the polygon
        path = mpl.path.Path(cell_polygon)
        mask = path.contains_points(grid_coords)
        if mask_sum is not None:
            mask = np.logical_and(mask, ~mask_sum)
            mask_sum = np.logical_or(mask, mask_sum)
        else:
            mask_sum = mask
        reserves_list.append(u_field_grades[mask].sum() * conv_factor)
    return reserves_list

In [ ]:
# Compute the reserve associated to the cells in t
# Need to transpose for the display
true_reserves = get_cell_reserves(true_min_t_cinet.T.ravel(), polygons, dx, dy, dz)
estimated_reserves = get_cell_reserves(
    estimate_min_t_cinet.T.ravel(), polygons, dx, dy, dz
)

plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 6)},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True)
    },
)

for ax, reserves in zip(plotter.ax_dict.values(), (true_reserves, estimated_reserves)):
    for i, j in inj_grid_locations:
        ax.plot(i, j, "ko")

    for i, j in prod_grid_locations:
        ax.plot(i, j, "ro")

    patches = []
    for i, cell_polygon in enumerate(polygons):
        # Add the polygon to the collection of patches
        xy = np.array(cell_polygon)
        patches.append(Polygon(xy, closed=True, facecolor=None))

        centroid = (xy.mean(axis=0)[0], xy.mean(axis=0)[1] * 1.05)
        ax.text(*centroid, f"{reserves[i]:.2f} t", fontsize=14, fontweight="bold")

        # Plot the number of the polygon
    p = PatchCollection(patches, alpha=0.75, cmap=plt.get_cmap("jet"))
    # p.set_facecolors("white")
    p.set_edgecolors("darkgrey")
    p.set_linewidth(0.6)
    p.set_linestyle("-")
    p.set_array(np.array(reserves, dtype=float))
    # p.set_label(np.array(np.round(reserves, 2), dtype=float))
    p.set_clim(2, 6)

    ax.add_collection(p)

# Add a common colorbar
plotter.fig.colorbar(p, label="Uranium [t]")

# Box dimensions
for ax in plotter.ax_dict.values():
    ax.set_xlim(0, nx * dx)
    ax.set_ylim(0, ny * dy)
    ax.set_aspect("equal", adjustable="box")

plotter.subfigs["fig0"].suptitle(
    "Metal per production cell [t]", fontweight="bold", fontsize=18
)
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

fname = "reserves_per_cells"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Code for benchmark

In [ ]:
# def generate_data(results, params, element):

#     data = initiate_data()

#     for scenario in sorted(params['simulations'].keys()):
#         data['scenarios'].append(scenario)
#         # wells
#         collection = results['producer_collections'][scenario]
#         rmse = get_asset_collection_RMSE_sum(collection, element)
#         data['sum of producers RMSE'].append(rmse)
#         # cells
#         collection = results['HM_cell_collections'][scenario]
#         rmse = get_asset_collection_RMSE_sum(collection, element)
#         data['sum of cells RMSE'].append(rmse)
#         # blocks
#         collection = results['HM_block_collections'][scenario]
#         rmse = get_asset_collection_RMSE_sum(collection, element)
#         data['sum of blocks RMSE'].append(rmse)
#         # zone
#         collection = results['HM_zone_collections'][scenario]
#         rmse = get_asset_collection_RMSE_sum(collection, element)
#         data['zone RMSE'].append(rmse)

#     # weighted rmse
#     w_rmse  = (0.3 * np.array(data['zone RMSE'])
#                + 0.3 * np.array(data['sum of blocks RMSE'])
#                + 0.3 * np.array(data['sum of cells RMSE']))
#     data['weighted RMSE'] = w_rmse

#     return data

- Creation of mineral files

In [ ]:
index = np.arange(nx * ny)

data = pd.DataFrame(
    data={
        "node-number": index,
        "Min_T_Cinet": true_min_t_cinet.T.ravel(),
    },  # need to flatten the parameter
    index=index,
)
simu_true.add_param_file_data(ParameterFiles.MINERALS, data)

data = pd.DataFrame(
    data={
        "node-number": index,
        "Min_T_Cinet": estimate_min_t_cinet.T.ravel(),
    },  # need to flatten the parameter
    index=index,
)
simu_estimate.add_param_file_data(ParameterFiles.MINERALS, data)

- Write input files

In [ ]:
simu_true.write_input_files()
simu_estimate.write_input_files()

- Run simulations

In [ ]:
runner.run(simu_true)
runner.run(simu_estimate)

- Reading the results and displaying available fields

In [ ]:
simu_true.read_hytec_results()
simu_true.handlers.results.grid_res_columns

In [ ]:
simu_true.handlers.results.flux_res_columns

In [ ]:
simu_estimate.read_hytec_results()
simu_estimate.handlers.results.grid_res_columns

In [ ]:
simu_estimate.handlers.results.flux_res_columns

In [ ]:
# Get the coordinates so we can plot the velocities
x_grid = simu_true.handlers.results.extract_field_from_grid_res(
    field="x-distance", nx=nx, ny=ny
)
y_grid = simu_true.handlers.results.extract_field_from_grid_res(
    field="y-distance", nx=nx, ny=ny
)

# Get heads and velocities
fwd_heads_grid_true_hytec = simu_true.handlers.results.extract_field_from_grid_res(
    field="head [m]", nx=nx, ny=ny
)

x_velocities = simu_true.handlers.results.extract_field_from_grid_res(
    field="x-flowrate [m/s]", nx=nx, ny=ny
)
y_velocities = simu_true.handlers.results.extract_field_from_grid_res(
    field="y-flowrate [m/s]", nx=nx, ny=ny
)

# Get the results on a 3D grid with the last dimension as time step
fwd_conc_grid_true_hytec = simu_true.handlers.results.extract_field_from_grid_res(
    field="T_Cinet [mol/l]", nx=nx, ny=ny
)
min_t_cinet_true_hytec = simu_true.handlers.results.extract_field_from_grid_res(
    field="Min_T_Cinet [mol/kg]", nx=nx, ny=ny
)
# Getting sample time. The unit is the same than the one defined for the simulation duration in the htc
grid_sample_times = simu_true.handlers.results.get_sample_times_from_grid_res()

# Get the results on a 3D grid with the last dimension as time step
fwd_conc_grid_estimate_hytec = (
    simu_estimate.handlers.results.extract_field_from_grid_res(
        field="T_Cinet [mol/l]", nx=nx, ny=ny
    )
)
min_t_cinet_estimate_hytec = simu_estimate.handlers.results.extract_field_from_grid_res(
    field="Min_T_Cinet [mol/kg]", nx=nx, ny=ny
)

- Check if the Min_T_Cinet field has been correctly updated in the HYTEC simulations

In [ ]:
plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (10, 5)},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True)
    },
)
plotter.plot_2d_field(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "True HYTEC": min_t_cinet_true_hytec[:, :, 0, 0] / conv_u,
        "Initial guess HYTEC": min_t_cinet_estimate_hytec[:, :, 0, 0] / conv_u,
    },
    cbar_title="Metal grade $[ppm]$",
    imshow_kwargs={"cmap": plt.get_cmap("jet"), "extent": [0.0, nx * dx, 0.0, ny * dy]},
    xlabel="X (m)",
    ylabel="Y (m)",
)
plotter.subfigs["fig0"].suptitle("Metal grade [ppm]", fontweight="bold")

fname = "Min_T_Cinet_field_true_vs_initial_estimation"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Plot the charges and the velocity: here should be zero everywhere

In [ ]:
plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (8, 4)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True),
    },
)

plotter.plot_2d_field(
    ax_names=["ax1-1"],
    data={
        "Head [m]": fwd_heads_grid_true_hytec[:, :, 0, -1],
    },
    cbar_title="Head $[m]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy]},
    # is_add_grid=True,
)

plotter.ax_dict["ax1-2"].quiver(
    x_grid[:, :, 0, -1],
    y_grid[:, :, 0, -1],
    x_velocities[:, :, 0, -1],
    y_velocities[:, :, 0, -1],
    color="C0",
    scale_units="xy",
)
plotter.ax_dict["ax1-2"].set_title("Darcy velocity [m2/s]")

plotter.subfigs["fig0"].suptitle("Hydro (HYTEC)", fontweight="bold")
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

fname = "T_Cinet_true_vs_initial_estimation"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Plot the initial concentration vs the final ones obtained with HYTEC:

In [ ]:
plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 4)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2", "ax1-3"]], sharey=True, sharex=True),
    },
)

plotter.plot_2d_field(
    ax_names=["ax1-1", "ax1-2", "ax1-3"],
    data={
        "Initial": fwd_conc_grid_true_hytec[:, :, 0, 0],
        "Final - True": fwd_conc_grid_true_hytec[:, :, 0, -1],
        "Final - Estimated": fwd_conc_grid_estimate_hytec[:, :, 0, -1],
    },
    cbar_title="Concentration $[molal]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy]},
    # is_add_grid=True,
)
plotter.subfigs["fig0"].suptitle("Tracer concentrations (HYTEC)", fontweight="bold")
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

fname = "T_Cinet_true_vs_initial_estimation"
for format in ["png", "pdf"]:
    plotter.savefig((fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Animation of the concentration evolution

In [ ]:
# Plot a concentration animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (8, 4)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True),
    },
)
nb_frames = nsamples

plotter.plot_2d_animated(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "True": fwd_conc_grid_true_hytec[:, :, 0, :],
        "Estimated": fwd_conc_grid_estimate_hytec[:, :, 0, :],
    },
    cbar_title="Concentration $[molal]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy]},
    nb_frames=nb_frames,
    # is_add_grid=True,
)
plotter.subfigs["fig0"].suptitle("Tracer concentration HYTEC", fontweight="bold")
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

plotter.close()
plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("fwd_conc_2d_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(fname_html, writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Animation of the Min_T_Cinet mineral grade evolution

In [ ]:
# Plot a concentration animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (8, 4)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True),
    },
)
nb_frames = nsamples

plotter.plot_2d_animated(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "True": min_t_cinet_true_hytec[:, :, 0, :] / conv_u,
        "Estimated": min_t_cinet_estimate_hytec[:, :, 0, :] / conv_u,
    },
    cbar_title="Metal grade $[ppm]$",
    imshow_kwargs={"cmap": plt.get_cmap("jet"), "extent": [0.0, nx * dx, 0.0, ny * dy]},
    nb_frames=nb_frames,
    # is_add_grid=True,
)
plotter.subfigs["fig0"].suptitle("Min_T_Cinet grade evolution HYTEC", fontweight="bold")
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")


plotter.close()
plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("fwd_GRADE_2d_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(fname_html, writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Plot the concentrations at the producer wells. Note, we add some white noise to the "True" data :)

In [ ]:
rng = np.random.default_rng(2021)
noise_std = 5e-5  # This is an absolute value


def make_noisy(x: NDArrayFloat) -> NDArrayFloat:
    """Return the input with some added white noise.

    Note
    ----
    The parameters are hardcoded to be consistent in the notebook.
    Change the function directly.
    """
    mean_noise = 0.0  # mean
    return x + rng.normal(mean_noise, noise_std, x.shape)

In [ ]:
# Indicate the order in which to plot
obs_plot_locations = ["ax2-3", "ax1-2", "ax3-2", "ax2-2", "ax1-1", "ax3-1", "ax2-1"]

plotter = FieldPlotter(
    fig_params={"constrained_layout": True, "figsize": (12, 8)},
    subfigs_params={
        "ncols": 3  # np.unique(np.array(prod_grid_locations)[:, 0].round()).size,
    },
    subplots_mosaic_params={
        "left_col": dict(
            mosaic=[["ghost1"], ["ax1-1"], ["ax1-2"], ["ghost2"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
        "centered_col": dict(
            mosaic=[["ax2-1"], ["ax2-2"], ["ax2-3"]], sharey=True, sharex=True
        ),
        "right_col": dict(
            mosaic=[["ghost3"], ["ax3-1"], ["ax3-2"], ["ghost4"]],
            gridspec_kw=dict(
                height_ratios=[1.0, 1.2, 1.2, 1.0],
            ),
            sharey=True,
            sharex=True,
        ),
    },
)

# hide axes and borders
plotter.ax_dict["ghost1"].axis("off")
plotter.ax_dict["ghost2"].axis("off")
plotter.ax_dict["ghost3"].axis("off")
plotter.ax_dict["ghost4"].axis("off")

# Static plot
count = 0  # to avoid unbounded linting
for count, ((x, y), ax_name) in enumerate(zip(prod_grid_locations, obs_plot_locations)):
    ix = int(x // dx)
    iy = int(y // dy)
    # obs_well_name = f"obs. well @ node #{ix}-{jx} \n (x={x:.1f}m) (y={y:.1f}m)"
    obs_well_name = f"obs. well @ \n x={x:.1f}m, y={y:.1f}m"

    plotter.plot_1d_field(
        ax_name=ax_name,
        title=obs_well_name,
        data={
            "True (noisy)": {
                # Note: we add some white noise
                "data": np.abs(make_noisy(fwd_conc_grid_true_hytec[ix, iy, 0, :])),
                "kwargs": {"marker": "o", "linestyle": "None", "c": "b", "alpha": 0.5},
            },
            "Initial estimation": {
                "data": fwd_conc_grid_estimate_hytec[ix, iy, 0, :],
                "kwargs": {"c": "r"},
            },
        },
        xlabel="Time [d]",
        ylabel="T_Cinet [mol/s]",
    )

ymax = max([ax.get_ylim()[1] for ax in plotter.ax_dict.values()])
for ax in plotter.ax_dict.values():
    ax.set_ylim(-0.1 * ymax, ymax)

plotter.add_fig_legend(ncol=2)
plotter.fig.supxlabel("Time [d]", fontweight="bold")
plotter.fig.supylabel("T_Cinet [mol/s]", fontweight="bold")

fname = "T_Cinet_true_at_producers"
for format in ["png", "pdf"]:
    plotter.savefig(str(fig_save_path.joinpath(f"{fname}.{format}")), format=format)

- Compute the objective function

In [ ]:
fun = 0
for x, y in prod_grid_locations:
    ix = int(x // dx)
    iy = int(y // dy)
    fun += np.sum(
        np.square(
            np.abs(make_noisy(fwd_conc_grid_true_hytec[ix, iy, 0, :]))
            - fwd_conc_grid_estimate_hytec[ix, iy, 0, :]
        )
    )
fun = fun / noise_std / noise_std

print("J0 = ", fun)

In [ ]:
grid_sample_times

## Inversion

- Creation of a base simulation for inversion with observation files, and the associated zones in the htc file.

In [ ]:
simu_inverse = copy.deepcopy(simu_estimate)
simu_inverse.update_root_and_name(new_root="simu_inverse", new_name="simu_inverse")

simu_inverse.handlers.htc.samples = nsamples

# 1) Add the observation wells
for count, (x, y) in enumerate(prod_grid_locations):
    ix = int(x // dx)
    iy = int(y // dy)

    name = f"well_{ix}_{iy}"
    zone = HytecZone(name, htc_path=simu_inverse.htc_file_path)
    zone.geometries.append(f"rectangle {x},{y}, {dx},{dy}")
    if fwd_conc_grid_true_hytec[ix, iy, 0, 0] < 1e-5:
        zone.geochem = "chem_base"
    else:
        zone.geochem = "chem_center"

    # Add the observables for the area
    obs = Observable(
        zone_name=zone.name,
        column_index=3,
        root_relative_path=f"observables/well{count + 1}_o.dat",
        root_path=simu_inverse.root,
        type=ObservationType.GRID,
        uncertainties_column_index=4,
    )

    vals = -np.abs(make_noisy(fwd_conc_grid_true_hytec[ix, iy, 0, :]))
    odata = pd.DataFrame(
        data={
            "time [s]": grid_sample_times,
            "node-number": indices_to_hytec_node_number(ix, nx=nx, iy=iy),
            "T_Cinet [mol/s]": vals,
            "T_Cinet-uncertainty [mol/s]": noise_std,
        }
    )
    obs.handler.update_data(odata)
    zone.observables.append(obs)

    simu_inverse.handlers.htc.zones_dict[name] = zone


# 2) Update the htc file with some options
simu_inverse.handlers.htc.optimization = "enabled"
simu_inverse.handlers.htc.optimization_solver = OptimizationSolverConfig(
    solver_name="lbfgsb",
    max_number_iterations=30,
    max_number_fwd_model_eval=150,
    max_number_gradient_eval=2,
    max_number_hessian_eval=1,
    objfun_threshold=1e-1,
    objfun_min_change=0.0,
    param_min_change=0.0,
    gradient_min_norm=0.0,
    hessian_max_norm=0.0,
    grad_history_size=5,
    adjoint_state="enabled",
    fd_gradient_check="enabled",
)
simu_inverse.handlers.htc.adjusted_parameters_dict = {
    "Min_T_Cinet": AdjustedParameterConfig(
        name="mineral Min_T_Cinet",
        lbounds=0.0,
        ubounds=800 * conv_u,  # 700 ppm
    )
}
simu_inverse.handlers.htc.adjoint_sampling = (
    nsamples  # number of samples on the adjoint variables
)

# 3)Write the input files
simu_inverse.write_input_files()

# 4) Run !
runner.run(simu_inverse)

# 5) Read the results
simu_inverse.read_hytec_results()

print("objective function J0 = ", simu_inverse.handlers.results.optim_res.obj_funs[0])
print("vmult = ", 1 / simu_inverse.handlers.results.optim_res.obj_funs[0])

In [ ]:
print("objective function J0 = ", simu_inverse.handlers.results.optim_res.obj_funs[0])
print("vmult = ", 1 / simu_inverse.handlers.results.optim_res.obj_funs[0])

- Read the results

In [ ]:
simu_inverse.read_hytec_results()

- Extract the adjoint variables computed for the first gradient

In [ ]:
print(f"columns = {simu_inverse.handlers.results.optim_res.adjoint_var_columns}")

In [ ]:
adj_conc_hytec = simu_inverse.handlers.results.extract_field_from_adj_var_res(
    field="adjoint-variable{T_Cinet} [m]", nx=nx, ny=ny
)
adj_min_hytec = simu_inverse.handlers.results.extract_field_from_adj_var_res(
    field="adjoint-variable{T_Cinet} [m]", nx=nx, ny=ny
)
adj_conc_hytec.shape

- Plot the adjoint variable evolution

In [ ]:
# Plot a concentration animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (8, 4)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True),
    },
)
nb_frames = nsamples

plotter.plot_2d_animated(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "Adj T_Cinet": adj_conc_hytec[:, :, 0, ::-1],
        "Adj_Min_T_Cinet": adj_min_hytec[:, :, 0, ::-1],
    },
    cbar_title="Concentration $[molal]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy]},
    nb_frames=nb_frames,
    # is_add_grid=True,
)
plotter.subfigs["fig0"].suptitle("Adjoint variables (conc and min)", fontweight="bold")
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")

plotter.close()
plotter.animate(nb_frames=nb_frames)
# Save the animation locally on the computer
fname_html = fig_save_path.joinpath("adj_variables_2d_animation.html")
writer = HTMLWriter(fps=5, embed_frames=True)
writer.frame_format = "svg"  # Ensure svg format
plotter.animation.save(fname_html, writer=writer)

# Extract the svg from the html file (for animation in Latex)
ngp.extract_frames_from_embedded_html_animation(fname_html)

# Display the animation
HTML(fname_html.read_text())

- Extract and plot the gradient

In [ ]:
# Get the HYTEC  Adj gradient
hytec_fd_gradients = simu_inverse.handlers.results.optim_res.fd_gradients
hytec_adjoint_gradient = simu_inverse.handlers.results.optim_res.adjoint_gradients

In [ ]:
grad_values: NDArrayFloat = (
    hytec_adjoint_gradient[0].loc[:, "value"].to_numpy().reshape(ny, nx)
)

In [ ]:
# Plot a concentration animation Here !
plotter = ngp.AnimatedPlotter(
    fig_params={"constrained_layout": True, "figsize": (8, 4)},
    subfigs_params={"nrows": 1},
    subplots_mosaic_params={
        "fig0": dict(mosaic=[["ax1-1", "ax1-2"]], sharey=True, sharex=True),
    },
)

plotter.plot_2d_static(
    ax_names=["ax1-1", "ax1-2"],
    data={
        "Grad": grad_values,
        "Grad2": grad_values,
    },
    cbar_title="Concentration $[molal]$",
    imshow_kwargs={"extent": [0.0, nx * dx, 0.0, ny * dy]},
    # is_add_grid=True,
)
plotter.subfigs["fig0"].suptitle("Adjoint variables (conc and min)", fontweight="bold")
plotter.subfigs["fig0"].supxlabel("X (m)", fontweight="bold")
plotter.subfigs["fig0"].supylabel("Y (m)", fontweight="bold")